In [66]:
import os
import hsrb_interface
import rospy
import sys
from hsrb_interface import geometry

In [67]:
os.environ['ROS_MASTER_URI'] = 'http://hsrb.local:11311'
os.environ['ROS_IP'] = '169.254.175.178'  // IP address of your laptop
print(os.environ['ROS_MASTER_URI'])
print(os.environ['ROS_IP'])

http://hsrb.local:11311
169.254.175.178


In [68]:
# Move timeout[s]
_MOVE_TIMEOUT=60.0
# Grasp force[N]
_GRASP_FORCE=0.2
# TF name of the OBJECT
_BOTTLE_TF='ar_marker/4'
_TABLE_TF='ar_marker/6'
_HOMEPOS_TF='ar_marker/3'
_TRASH_TF='ar_marker/7'
# TF name of the gripper
_HAND_TF='hand_palm_link'

In [69]:
# Preparation for using the robot functions
robot = hsrb_interface.Robot()
omni_base = robot.get('omni_base')
whole_body = robot.get('whole_body')
gripper = robot.get('gripper')
tts = robot.get('default_tts')

In [70]:
# Posture that 0.02[m] front and rotate -1.57 around z-axis of the bottle maker
bottle_to_hand = geometry.pose(z=-0.02, ek=-1.57)
bottle_to_trash = geometry.pose(z=-0.2, ek=-1.57)

# Posture to move the hand 0.1[m] up
hand_up = geometry.pose(x=0.1)

# Posture to move the hand 0.5[m] back
hand_back = geometry.pose(z=-0.5)

# Location of the sofa
sofa_pos = (1.2, 0.4, 1.57)

In [71]:
# Greet
rospy.sleep(3.0)
whole_body.move_to_go()
tts.say('こんにちは. My name is HSR. Nice to meet you. はじめましょう.')
rospy.sleep(5.0)

In [72]:
omni_base.pose

[0.1996876919000958, 0.2155823133246017, -0.20140952605014725]

In [73]:
# Transit to initial grasping posture
whole_body.move_to_neutral()
# Look at the hand after the transition
whole_body.looking_hand_constraint = True

In [75]:
try:
     # Transit to initial grasping posture
    whole_body.move_to_go()
    omni_base.go_abs(2.598127270382162, 0.24041103105294316, -0.16538801251765786, 300.0)
    whole_body.move_to_neutral()
    rospy.sleep(1.0)
    omni_base.go_pose(geometry.pose(z=-1.0, ei=3.14, ej=-1.57), 100.0, ref_frame_id='ar_marker/6')
    tts.say('I will scan the bottle')
    omni_base.pose
except:
    tts.say('たすけてください')
    rospy.logerr('fail to init')
    sys.exit()

In [76]:
omni_base.pose #table

[2.7373663498903436, 0.2271743743052535, -0.2532782648420947]

In [77]:
try:
    rospy.sleep(2.0)
    whole_body.move_to_neutral()
    # Look at the hand after the transition
    whole_body.looking_hand_constraint = True
    # Move the hand to front of the bottle
    whole_body.move_end_effector_pose(bottle_to_hand, _BOTTLE_TF)
    # Specify the force to grasp
    gripper.apply_force(_GRASP_FORCE)
    # Wait time for simulator's grasp hack. Not needed on actual robot
    rospy.sleep(2.0)
    # Move the hand up on end effector coordinate
    whole_body.move_end_effector_pose(hand_up, _HAND_TF)
    # Move the hand back on end effector coordinate
    whole_body.move_end_effector_pose(hand_back, _HAND_TF)
    # Transit to initial posture
    whole_body.move_to_neutral()
except:
    tts.say('たすけてください')
    rospy.logerr('fail to init')
    sys.exit()

In [78]:
try:
    omni_base.go_abs(1.6889372569076162, -1.4412442913589325, -1.528230570580849, 300.0)
    tts.say('I will put the bottle into the trash')
    rospy.sleep(5.0)
    whole_body.move_to_neutral()
    #Look at the hand after the transition
    whole_body.looking_hand_constraint = True
    # Move the hand to front of the bottle
    whole_body.move_end_effector_pose(bottle_to_trash, _TRASH_TF)
    gripper.command(1.2)
    omni_base.pose
    whole_body.move_end_effector_pose(hand_back, _HAND_TF)
    whole_body.move_to_go()
    omni_base.go_rel(0.0, 0.0, 2.5, 100.0)

except:
    try:
        omni_base.go_rel(0.0, 0.0, 0.7, 100.0)
        tts.say('I will put the bottle into the trash')
        rospy.sleep(5.0)
        #omni_base.go_rel(1.5, 0.0, 0.0, 100.0)
        whole_body.move_to_neutral()
        #Look at the hand after the transition
        whole_body.looking_hand_constraint = True
        # Move the hand to front of the bottle
        whole_body.move_end_effector_pose(bottle_to_trash, _TRASH_TF)
        gripper.command(1.2)
        whole_body.move_end_effector_pose(hand_back, _HAND_TF)
        whole_body.move_to_go()
        omni_base.go_rel(0.0, 0.0, 2.5, 100.0)
    except:
        omni_base.go_rel(0.0, 0.0, -0.5, 100.0)
        tts.say('I will put the bottle into the trash')
        rospy.sleep(5.0)
        #omni_base.go_rel(1.5, 0.0, 0.0, 100.0)
        whole_body.move_to_neutral()
        #Look at the hand after the transition
        whole_body.looking_hand_constraint = True
        # Move the hand to front of the bottle
        whole_body.move_end_effector_pose(bottle_to_trash, _TRASH_TF)
        gripper.command(1.2)
        whole_body.move_end_effector_pose(hand_back, _HAND_TF)
        whole_body.move_to_go()
        omni_base.go_rel(0.0, 0.0, 5.5, 100.0)
        tts.say('助けてください')
        rospy.logerr('fail to grasp')
        sys.exit()

In [79]:
omni_base.pose

[1.8459472666886823, -1.5672957195346993, 0.9950632838061805]

In [83]:
    #omni_base.go_rel(0.0, 0.0, 1.0, 100.0)
    omni_base.go_abs(-0.000896763500183281, 0.0005740724114742323, -0.0015563745280494801, 300.0)
    whole_body.move_to_go()
    tts.say('おわりました。ありがとうございました')
    whole_body.move_to_joint_positions({'head_tilt_joint': 1.0,'head_tilt_joint': -0.5})
    
    

MobileBaseError: Failed to reach goal ()

In [81]:
try:
    omni_base.go_rel(1.5, 0.1, 0.0, 100.0)
    tts.say('I am going home')
    rospy.sleep(3.0)
    omni_base.go_rel(0.0, 0.0, 1.0, 100.0)
    whole_body.move_to_neutral()
    omni_base.go_rel(2.0, 0.0, 0.0, 100.0)
    whole_body.move_to_go()
    omni_base.go_pose(geometry.pose(z=-0.5, ei=3.14, ej=-1.57), 100.0, ref_frame_id='ar_marker/3')
    rospy.sleep(2.0)
    omni_base.go_rel(0.0, 0.0, 3.5, 100.0)
    whole_body.move_to_go()
    whole_body.move_to_joint_positions({'head_tilt_joint': 1.0,'head_tilt_joint': -0.5})
    tts.say('おわりました。ありがとうございました')
    

except:
    try:
        omni_base.go_rel(0.0, 0.0, -1.0, 100.0)
        whole_body.move_to_neutral()
        omni_base.go_rel(0.5, 0.0, 0.0, 100.0)
        whole_body.move_to_go()
        omni_base.go_pose(geometry.pose(z=-0.5, ei=3.14, ej=-1.57), 100.0, ref_frame_id='ar_marker/3')
        rospy.sleep(2.0)
        omni_base.go_rel(0.0, 0.0, 3.5, 100.0)
        whole_body.move_to_go()
        whole_body.move_to_joint_positions({'head_tilt_joint': 1.0,'head_tilt_joint': -0.5})
    except:
        omni_base.go_rel(0.0, 0.0, 1.0, 100.0)
        whole_body.move_to_neutral()
        omni_base.go_rel(0.5, 0.0, 0.0, 100.0)
        whole_body.move_to_go()
        omni_base.go_pose(geometry.pose(z=-0.5, ei=3.14, ej=-1.57), 100.0, ref_frame_id='ar_marker/3')
        rospy.sleep(2.0)
        omni_base.go_rel(0.0, 0.0, 3.5, 100.0)
        whole_body.move_to_go()
        whole_body.move_to_joint_positions({'head_tilt_joint': 1.0,'head_tilt_joint': -0.5})
    tts.say('おわりました。ありがとうございました')
    tts.say('助けてください')
    rospy.logerr('fail to grasp')
    sys.exit()

In [ ]:
%%bash
rosrun rviz rviz  -d rospack find hsrb_common_launch/config/hsrb_display_full_hsrb.rviz